# فیلترهای غیرخطی / Non-Linear Filters

**هدف:** یادگیری فیلترهای غیرخطی برای حذف نویز نمک و فلفل

---

## محتوا:
1. نویز نمک و فلفل (Salt & Pepper)
2. فیلتر میانه (Median Filter)
3. فیلتر دوطرفه (Bilateral Filter)
4. مقایسه با فیلترهای خطی

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12

print(f"OpenCV: {cv2.__version__}")

## 1. توابع کمکی

In [ ]:
def add_salt_pepper_noise(image: np.ndarray, salt_prob: float = 0.01, pepper_prob: float = 0.01) -> np.ndarray:
    """
    اضافه کردن نویز نمک و فلفل
    
    Args:
        image: تصویر ورودی
        salt_prob: احتمال نویز سفید (نمک)
        pepper_prob: احتمال نویز سیاه (فلفل)
    """
    noisy = image.copy()
    
    # نویز نمک (سفید)
    salt_mask = np.random.random(image.shape) < salt_prob
    noisy[salt_mask] = 255
    
    # نویز فلفل (سیاه)
    pepper_mask = np.random.random(image.shape) < pepper_prob
    noisy[pepper_mask] = 0
    
    return noisy


def calculate_psnr(original: np.ndarray, processed: np.ndarray) -> float:
    """محاسبه PSNR"""
    mse = np.mean((original.astype(np.float64) - processed.astype(np.float64)) ** 2)
    if mse == 0:
        return float('inf')
    return 10 * np.log10(255**2 / mse)


def display_images(images: list, titles: list, cmap='gray'):
    """نمایش تصاویر"""
    n = len(images)
    fig, axes = plt.subplots(1, n, figsize=(5*n, 5))
    if n == 1:
        axes = [axes]
    for ax, img, title in zip(axes, images, titles):
        ax.imshow(img, cmap=cmap)
        ax.set_title(title, fontsize=14, fontweight='bold')
        ax.axis('off')
    plt.tight_layout()
    plt.show()

print("✓ توابع آماده شدند")

## 2. ایجاد تصویر و اضافه کردن نویز

In [ ]:
# ایجاد تصویر نمونه
height, width = 256, 256
original = np.zeros((height, width), dtype=np.uint8)

# اضافه کردن اشکال
cv2.rectangle(original, (50, 50), (150, 150), 200, -1)
cv2.circle(original, (200, 200), 40, 150, -1)
cv2.line(original, (0, 0), (255, 255), 100, 2)

# اضافه کردن نویز نمک و فلفل
noisy_sp = add_salt_pepper_noise(original, salt_prob=0.02, pepper_prob=0.02)

display_images(
    [original, noisy_sp],
    ['تصویر اصلی', 'نویز نمک و فلفل (4%)']
)

## 3. مشکل فیلترهای خطی با نویز نمک و فلفل

**چرا فیلترهای خطی کار نمی‌کنند؟**

فیلترهای خطی (میانگین، گاوسی) نویز را پخش می‌کنند به جای حذف کردن!

In [ ]:
# تست فیلترهای خطی
avg_filtered = cv2.blur(noisy_sp, (5, 5))
gaussian_filtered = cv2.GaussianBlur(noisy_sp, (5, 5), 1.0)

psnr_avg = calculate_psnr(original, avg_filtered)
psnr_gauss = calculate_psnr(original, gaussian_filtered)

print(f"PSNR فیلتر میانگین: {psnr_avg:.2f} dB")
print(f"PSNR فیلتر گاوسی: {psnr_gauss:.2f} dB")

display_images(
    [original, noisy_sp, avg_filtered, gaussian_filtered],
    ['اصلی', 'نویزی', 'فیلتر میانگین', 'فیلتر گاوسی']
)

print("\n⚠️ توجه: فیلترهای خطی نویز را محو می‌کنند اما حذف نمی‌کنند!")

## 4. فیلتر میانه (Median Filter)

**تئوری:**
- به جای میانگین، میانه (median) پیکسل‌های همسایه را محاسبه می‌کند
- میانه نسبت به outlier مقاوم است
- برای نویز نمک و فلفل عالی است

**مثال 3×3:**
```
همسایگان: [0, 0, 100, 100, 100, 100, 100, 255, 255]
میانگین: 117
میانه: 100  ← بهتر!
```

In [ ]:
# فیلتر میانه با اندازه‌های مختلف
kernel_sizes = [3, 5, 7]
median_filtered = []

for ksize in kernel_sizes:
    filtered = cv2.medianBlur(noisy_sp, ksize)
    median_filtered.append(filtered)
    
    psnr = calculate_psnr(original, filtered)
    print(f"Median {ksize}×{ksize}: PSNR = {psnr:.2f} dB")

# نمایش
images = [original, noisy_sp] + median_filtered
titles = ['اصلی', 'نویزی'] + [f'Median {k}×{k}' for k in kernel_sizes]
display_images(images, titles)

## 5. مقایسه: میانگین vs میانه

In [ ]:
# مقایسه مستقیم
ksize = 5
avg_result = cv2.blur(noisy_sp, (ksize, ksize))
median_result = cv2.medianBlur(noisy_sp, ksize)

psnr_avg = calculate_psnr(original, avg_result)
psnr_median = calculate_psnr(original, median_result)

print("="*50)
print(f"فیلتر میانگین {ksize}×{ksize}: PSNR = {psnr_avg:.2f} dB")
print(f"فیلتر میانه {ksize}×{ksize}:   PSNR = {psnr_median:.2f} dB")
print(f"بهبود: {psnr_median - psnr_avg:.2f} dB")
print("="*50)

display_images(
    [original, noisy_sp, avg_result, median_result],
    ['اصلی', 'نویزی', f'میانگین {ksize}×{ksize}', f'میانه {ksize}×{ksize}']
)

## 6. فیلتر دوطرفه (Bilateral Filter)

**تئوری:**
- فیلتر گاوسی که لبه‌ها را حفظ می‌کند
- دو وزن دارد:
  1. وزن فضایی (spatial): فاصله از مرکز
  2. وزن شدت (intensity): تفاوت مقدار پیکسل
- اگر پیکسل همسایه خیلی متفاوت باشد (لبه)، وزن کمی می‌گیرد

**کاربرد:** هموارسازی بدون از دست دادن جزئیات

In [ ]:
# اضافه کردن نویز گاوسی (برای تست bilateral)
def add_gaussian_noise(image, sigma=20):
    noise = np.random.normal(0, sigma, image.shape).astype(np.int16)
    noisy = image.astype(np.int16) + noise
    return np.clip(noisy, 0, 255).astype(np.uint8)

noisy_gaussian = add_gaussian_noise(original, sigma=25)

# فیلتر دوطرفه با پارامترهای مختلف
# d: قطر همسایگی
# sigmaColor: فیلتر در فضای رنگ
# sigmaSpace: فیلتر در فضای مختصات

bilateral_1 = cv2.bilateralFilter(noisy_gaussian, d=9, sigmaColor=75, sigmaSpace=75)
bilateral_2 = cv2.bilateralFilter(noisy_gaussian, d=9, sigmaColor=150, sigmaSpace=150)

# مقایسه با گاوسی
gaussian = cv2.GaussianBlur(noisy_gaussian, (9, 9), 2.0)

psnr_gauss = calculate_psnr(original, gaussian)
psnr_bi1 = calculate_psnr(original, bilateral_1)
psnr_bi2 = calculate_psnr(original, bilateral_2)

print(f"Gaussian: PSNR = {psnr_gauss:.2f} dB")
print(f"Bilateral (σc=75, σs=75): PSNR = {psnr_bi1:.2f} dB")
print(f"Bilateral (σc=150, σs=150): PSNR = {psnr_bi2:.2f} dB")

display_images(
    [original, noisy_gaussian, gaussian, bilateral_1, bilateral_2],
    ['اصلی', 'نویز گاوسی', 'Gaussian', 'Bilateral (75)', 'Bilateral (150)']
)

## 7. مقایسه کامل: همه فیلترها

In [ ]:
# تست روی نویز نمک و فلفل
print("\n" + "="*60)
print("مقایسه فیلترها برای نویز نمک و فلفل:")
print("="*60)

noisy = add_salt_pepper_noise(original, 0.03, 0.03)

results = [
    ('نویزی', noisy),
    ('میانگین 5×5', cv2.blur(noisy, (5, 5))),
    ('گاوسی 5×5', cv2.GaussianBlur(noisy, (5, 5), 1.0)),
    ('میانه 5×5', cv2.medianBlur(noisy, 5)),
    ('Bilateral', cv2.bilateralFilter(noisy, 9, 75, 75))
]

for name, img in results:
    psnr = calculate_psnr(original, img)
    print(f"{name:20s} | PSNR: {psnr:6.2f} dB")

images = [original] + [img for _, img in results]
titles = ['اصلی'] + [name for name, _ in results]
display_images(images, titles)

## 8. تمرین عملی

**وظیفه:**
1. تصویر خود را بارگذاری کنید
2. نویز نمک و فلفل با احتمال 5% اضافه کنید
3. همه فیلترها را تست کنید
4. بهترین فیلتر را انتخاب کنید

In [ ]:
# کد تمرین شما
# TODO: بارگذاری تصویر
# TODO: اضافه کردن نویز
# TODO: تست فیلترها
# TODO: مقایسه PSNR

pass

## نتیجه‌گیری

### انتخاب فیلتر مناسب:

| نوع نویز | فیلتر پیشنهادی | دلیل |
|---------|----------------|------|
| گاوسی | Gaussian / Bilateral | هموارسازی خوب |
| نمک و فلفل | **Median** | حذف کامل نویز |
| ترکیبی | Median + Bilateral | ابتدا median، سپس bilateral |
| حفظ لبه | **Bilateral** | هموارسازی بدون از دست دادن جزئیات |

### نکات کلیدی:

1. **فیلتر میانه:**
   - ✓ عالی برای نویز نمک و فلفل
   - ✓ غیرخطی - نویز را حذف می‌کند نه محو
   - ✗ کندتر از فیلترهای خطی

2. **فیلتر دوطرفه:**
   - ✓ لبه‌ها را حفظ می‌کند
   - ✓ برای نویز گاوسی عالی
   - ✗ محاسباتی گران
   - ✗ برای نمک و فلفل مناسب نیست

3. **ترکیب فیلترها:**
   - می‌توان چند فیلتر را پشت سر هم اعمال کرد
   - مثال: Median → Bilateral

---

**تمام شد!** 🎉

حالا می‌توانید:
- نویز را تشخیص دهید
- فیلتر مناسب را انتخاب کنید
- کیفیت را با PSNR ارزیابی کنید